# Stock price collector

## Ordinary imports

In [1]:
from pandas_datareader import data
from pandas_datareader._utils import RemoteDataError
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import math
import csv
import re
from datetime import datetime

import os.path
from os import path
import time

# import sklearn
# import scipy

# from sklearn.naive_bayes import GaussianNB
# import lightgbm as lightgbm
# from xgboost import XGBClassifier
# import keras

# Get Started
# https://github.com/Refinitiv-API-Samples/Article.EikonDataAPI.DotNet.Library
# https://developers.refinitiv.com/en/api-catalog/eikon/eikon-data-api/quick-start
# API limits
# https://developers.refinitiv.com/en/api-catalog/eikon/eikon-data-api/documentation?content=49692&type=documentation_item
import eikon as ek 

from pylab import rcParams
plt.rcParams['figure.figsize'] = 16, 8

In [2]:
fontsize_reg = 12
fontsize_title = 16
from pylab import rcParams
rcParams['figure.facecolor'] = '1'
rcParams['figure.figsize'] = [8.0, 3.5]
rcParams['figure.dpi'] = 80
rcParams['savefig.dpi'] = 600

rcParams['font.size'] = 12
rcParams['legend.fontsize'] = 'large'
rcParams['figure.titlesize'] = 'large'

## Import custom function to collect news and prices, work with file system and modelling

In [3]:
import trade_news_predict_prices

# Some constants (path, random seeed, api key)

In [4]:
general_constants = pd.read_csv('general_constants.txt', sep=',', delimiter = ",", header='infer', index_col='variable_name')
general_constants

,value
variable_name,
random_seed,420
directory_path,C:/DAN/t_systems/trade_news
folder_name_headlines,data/backup_headlines/
folder_name_logger,data/logger/
folder_name_merged_news_and_prices,data/merged_news_and_prices/
folder_name_for_labelling,data/for_labelling/
folder_name_for_labeled_news_iter1,data/labeled_news_iter1/
set_app_key1,ccebd1ff788d4574818be93e35c4741925ac214e
set_app_key2,a8be043d709946238759b0c5cb34c8f90d4e6f2c


In [5]:
# Path to folder with directory is a constant value that stores in .txt
# You can change them in some_constants.txt file or define them on your own.
directory_path = str(general_constants.loc['directory_path'][0])
random_seed = int(general_constants.loc['random_seed'][0])
print(random_seed)
print(directory_path)

420
C:/DAN/t_systems/trade_news


# Set API key

In [6]:
# Set API key to work with thomson reuters data
# Make sure refiniv eikon terminal is running and not disconnected
print(general_constants.loc['set_app_key1'][0])
print(general_constants.loc['set_app_key2'][0])
ek.set_app_key(general_constants.loc['set_app_key2'][0])

ccebd1ff788d4574818be93e35c4741925ac214e
a8be043d709946238759b0c5cb34c8f90d4e6f2c


# Try collect trading prices data

In [7]:
df = ek.get_timeseries(
    ["GOOGL.OQ"],
    interval='minute',
    start_date="2021-01-01",
    end_date="2021-01-30",
    fields=['TIMESTAMP', 'VALUE', 'VOLUME', 'HIGH', 'LOW', 'OPEN', 'CLOSE', 'COUNT'],
)

df

GOOGL.OQ,VALUE,VOLUME,HIGH,LOW,OPEN,CLOSE,COUNT
Date,,,,,,,
2021-01-01 00:04:00,<NA>,2,1752.60,1752.45,1752.45,1752.60,2
2021-01-01 00:07:00,<NA>,50,1753.15,1753.08,1753.08,1753.15,2
2021-01-01 00:21:00,<NA>,1,1752.82,1752.82,1752.82,1752.82,1
2021-01-01 00:39:00,<NA>,15,1753.96,1753.96,1753.96,1753.96,1
2021-01-01 00:56:00,<NA>,27,1754.08,1754.00,1754.00,1754.08,2
...,...,...,...,...,...,...,...
2021-01-29 23:41:00,<NA>,2,1827.56,1824.19,1824.19,1827.56,2
2021-01-29 23:46:00,<NA>,1,1823.00,1823.00,1823.00,1823.00,1
2021-01-29 23:47:00,<NA>,20,1824.49,1823.96,1824.49,1823.96,12


In [13]:
# Read the file with all news texts to create a list with rics in it
from trade_news_predict_prices import get_name_of_latest_file_in_folder
latest_file = get_name_of_latest_file_in_folder(general_constants.loc['folder_name_headlines'][0])
all_headlines_df = pd.read_csv(latest_file, sep=';')
all_headlines_df['versionCreated'] = pd.to_datetime(all_headlines_df['versionCreated'])
print(all_headlines_df.shape, latest_file)

(399141, 6) data/backup_headlines/all_headlines_df_2021-03-13_v3.csv


In [14]:
# Create the file with rics from all news texts dataset
# (prices for companies with already collected news)
rics_list = list(all_headlines_df['ric'].value_counts().index)
print(len(rics_list), rics_list)

# rics_list = ['JPM.N', 'AMZN.OQ', 'FB.OQ', 'AAPL.OQ', 'MSFT.OQ', 'GOOGL.OQ', 'TSLA.OQ', 'BABA.N', 'PFE.N', 'RWEG.DE', 'VOWG_p.DE', 'BAC.N', 'NFLX.OQ', 'WMT.N', 'SIEGn.DE', 'LHAG.DE', 'DBKGn.DE', 'DIS.N', 'JNJ.N', 'INTC.OQ', 'DAIGn.DE', 'T.N', 'BMWG.DE', 'MRK.N', 'BAYGn.DE', 'VZ.N', 'SAPG.DE', 'EONGn.DE', 'KO.N', 'CBKG.DE', 'MA.N', 'ALVG.DE', 'NKE.N', 'TKAG.DE', 'DTEGn.DE', 'NVDA.OQ', 'V.N', 'CMCSA.OQ', 'BRKa.N', 'UNH.N', 'BASFn.DE', 'PG.N', 'HD.N', 'TMUS.OQ', 'PYPL.OQ', 'ADSGn.DE', 'DPWGn.DE', 'CONG.DE', 'LUMN.N', 'PSMGn.DE', 'MRCG.DE', 'IFXGn.DE', 'FREG.DE', 'HNKG_p.DE', 'FMEG.DE', 'ATUS.N', 'ADBE.OQ', 'CHTR.OQ', 'VNAn.DE', 'JCOM.OQ', 'HEIG.DE', 'O2Dn.DE', 'PDD.OQ', 'UTDI.DE', 'VG.OQ', 'FNTGn.DE', 'BEIG.DE', 'DRIG.DE', 'TIGO.OQ', 'BAND.OQ', 'TC1n.DE', 'IRDM.OQ', 'CABO.N', 'CCOI.OQ', 'USM.N', 'NFN.DE', 'E4CG.DE', 'PH6.F', 'IVV.P', 'VTI.P']

87 ['JPM.N', 'AMZN.OQ', 'FB.OQ', 'AAPL.OQ', 'MSFT.OQ', 'GOOGL.OQ', 'TSLA.OQ', 'BABA.N', 'PFE.N', 'RWEG.DE', 'VOWG_p.DE', 'BAC.N', 'NFLX.OQ', 'WMT.N', 'SIEGn.DE', 'LHAG.DE', 'DBKGn.DE', 'DIS.N', 'JNJ.N', 'XOM.N', 'INTC.OQ', 'DAIGn.DE', 'T.N', 'BMWG.DE', 'MRK.N', 'BAYGn.DE', 'CSCO.OQ', 'VZ.N', 'SAPG.DE', 'EONGn.DE', 'CRM.N', 'KO.N', 'CBKG.DE', 'MA.N', 'ALVG.DE', 'NKE.N', 'TKAG.DE', 'DTEGn.DE', 'NVDA.OQ', 'V.N', 'CMCSA.OQ', 'BRKa.N', 'UNH.N', 'BASFn.DE', 'PG.N', 'HD.N', 'TMUS.OQ', 'ABT.N', 'ABBV.N', 'PYPL.OQ', 'PEP.OQ', 'TMO.N', 'ADSGn.DE', 'DPWGn.DE', 'CONG.DE', 'LUMN.N', 'PSMGn.DE', 'MRCG.DE', 'IFXGn.DE', 'FREG.DE', 'HNKG_p.DE', 'FMEG.DE', 'ATUS.N', 'ADBE.OQ', 'CHTR.OQ', 'VNAn.DE', 'JCOM.OQ', 'HEIG.DE', 'O2Dn.DE', 'PDD.OQ', 'UTDI.DE', 'VG.OQ', 'FNTGn.DE', 'BEIG.DE', 'DRIG.DE', 'TIGO.OQ', 'BAND.OQ', 'TC1n.DE', 'IRDM.OQ', 'CABO.N', 'CCOI.OQ', 'USM.N', 'NFN.DE', 'E4CG.DE', 'PH6.F', 'IVV.P', 'VTI.P']


In [9]:
# Or we can collect prices of all USA companies
rics_usa_excel = pd.read_excel('C:/DAN/t_systems/trade_project/rics_usa_excel.xlsx')
rics_list_usa = list(rics_usa_excel['ric'])
len(rics_list_usa)

3803

In [10]:
# Or all foreign companies which are available to trade in SPb
rics_available_spb_excel = pd.read_excel('C:/DAN/t_systems/trade_project/ListingSecurityList.xlsx')
rics_list_available_spb = list(rics_available_spb_excel['ric'])
len(rics_list_available_spb)

1556

In [16]:
# Collect prices
from trade_news_predict_prices import get_timeseries_of_rics_to_folder
get_timeseries_of_rics_to_folder(
    list_of_rics=rics_list,
    folder_for_timeseries_file='data/price_timeseries/',
    timeseries_interval='daily',
    timeseries_start_date="2010-01-01",
    timeseries_end_date="2021-06-01",
    timeseries_is_adjusted=True,
    sleep_between_rics=0.1)

0 JPM.N, Attempt 1, NOT EMPTY, 
       , data/price_timeseries/price_timeseries_daily_adjusted_JPM.N.csv, 24/06/2021 18:21:22
1 AMZN.OQ, Attempt 1, NOT EMPTY, 
         , data/price_timeseries/price_timeseries_daily_adjusted_AMZN.OQ.csv, 24/06/2021 18:21:24
2 FB.OQ, Attempt 1, NOT EMPTY, 
       , data/price_timeseries/price_timeseries_daily_adjusted_FB.OQ.csv, 24/06/2021 18:21:25
3 AAPL.OQ, Attempt 1, NOT EMPTY, 
         , data/price_timeseries/price_timeseries_daily_adjusted_AAPL.OQ.csv, 24/06/2021 18:21:27
4 MSFT.OQ, Attempt 1, NOT EMPTY, 
         , data/price_timeseries/price_timeseries_daily_adjusted_MSFT.OQ.csv, 24/06/2021 18:21:28
5 GOOGL.OQ, Attempt 1, NOT EMPTY, 
          , data/price_timeseries/price_timeseries_daily_adjusted_GOOGL.OQ.csv, 24/06/2021 18:21:29
6 TSLA.OQ, Attempt 1, NOT EMPTY, 
         , data/price_timeseries/price_timeseries_daily_adjusted_TSLA.OQ.csv, 24/06/2021 18:21:30
7 BABA.N, Attempt 1, NOT EMPTY, 
        , data/price_timeseries/price_timeseries_dai

2021-06-24 18:23:24,757 P[10784] [MainThread 18996] Error with DRIG.DE: The user does not have permission for the requested data
2021-06-24 18:23:24,758 P[10784] [MainThread 18996] DRIG.DE: The user does not have permission for the requested data | 


ERROR, Exception. Request fails or if server returns an error.
Attempt 1, error with that attempt, 

2021-06-24 18:23:35,564 P[10784] [MainThread 18996] Error with DRIG.DE: The user does not have permission for the requested data
2021-06-24 18:23:35,565 P[10784] [MainThread 18996] DRIG.DE: The user does not have permission for the requested data | 


ERROR, Exception. Request fails or if server returns an error.
Attempt 2, error with that attempt, 

    !!! ERROR WITH THIS RIC: DRIG.DE daily 2010-01-01 2021-06-01 adjusted

75 TIGO.OQ, Attempt 1, NOT EMPTY, 
          , data/price_timeseries/price_timeseries_daily_adjusted_TIGO.OQ.csv, 24/06/2021 18:23:46
76 BAND.OQ, Attempt 1, NOT EMPTY, 
          , data/price_timeseries/price_timeseries_daily_adjusted_BAND.OQ.csv, 24/06/2021 18:23:49
77 TC1n.DE, Attempt 1, NOT EMPTY, 
          , data/price_timeseries/price_timeseries_daily_adjusted_TC1n.DE.csv, 24/06/2021 18:23:51
78 IRDM.OQ, Attempt 1, NOT EMPTY, 
          , data/price_timeseries/price_timeseries_daily_adjusted_IRDM.OQ.csv, 24/06/2021 18:23:52
79 CABO.N, Attempt 1, NOT EMPTY, 
         , data/price_timeseries/price_timeseries_daily_adjusted_CABO.N.csv, 24/06/2021 18:23:54
80 CCOI.OQ, Attempt 1, NOT EMPTY, 
          , data/price_timeseries/price_timeseries_daily_adjusted_CCOI.OQ.csv, 24/06/2021 18:23:55
81 USM.N, Attempt 1, NO

In [20]:
list_of_files_to_merge = os.listdir('data/price_timeseries/')
print(len(list_of_files_to_merge))

86


In [21]:
all_stock_prices = pd.DataFrame(
    data = [], 
    columns=[])

for file_now in list_of_files_to_merge:
    file_path = 'data/price_timeseries/' + file_now
    timeseries_df = pd.read_csv(file_path, sep=';')
    print(file_path, timeseries_df.shape)
    
    all_stock_prices = pd.concat([all_stock_prices, timeseries_df], ignore_index=True)
    time.sleep(0)

data/price_timeseries/price_timeseries_daily_adjusted_AAPL.OQ.csv (2872, 9)
data/price_timeseries/price_timeseries_daily_adjusted_ABBV.N.csv (2133, 9)
data/price_timeseries/price_timeseries_daily_adjusted_ABT.N.csv (2872, 9)
data/price_timeseries/price_timeseries_daily_adjusted_ADBE.OQ.csv (2872, 9)
data/price_timeseries/price_timeseries_daily_adjusted_ADSGn.DE.csv (2892, 9)
data/price_timeseries/price_timeseries_daily_adjusted_ALVG.DE.csv (2892, 9)
data/price_timeseries/price_timeseries_daily_adjusted_AMZN.OQ.csv (2872, 9)
data/price_timeseries/price_timeseries_daily_adjusted_ATUS.N.csv (992, 9)
data/price_timeseries/price_timeseries_daily_adjusted_BABA.N.csv (1686, 9)
data/price_timeseries/price_timeseries_daily_adjusted_BAC.N.csv (2872, 9)
data/price_timeseries/price_timeseries_daily_adjusted_BAND.OQ.csv (893, 9)
data/price_timeseries/price_timeseries_daily_adjusted_BASFn.DE.csv (2892, 9)
data/price_timeseries/price_timeseries_daily_adjusted_BAYGn.DE.csv (2892, 9)
data/price_timeser

In [22]:
all_stock_prices

,VALUE,VOLUME,HIGH,LOW,OPEN,CLOSE,COUNT,ric,Date
0,NaN,1.525399e+08,7.660707,7.586421,7.626778,7.649278,NaN,AAPL.OQ,2010-01-04
1,NaN,1.704628e+08,7.699635,7.616421,7.671064,7.656421,NaN,AAPL.OQ,2010-01-05
2,NaN,1.879475e+08,7.685707,7.526778,7.655707,7.534635,NaN,AAPL.OQ,2010-01-06
3,NaN,1.456166e+08,7.571421,7.466421,7.559992,7.520707,NaN,AAPL.OQ,2010-01-07
4,NaN,1.410746e+08,7.571421,7.466421,7.517492,7.570707,NaN,AAPL.OQ,2010-01-08
...,...,...,...,...,...,...,...,...,...
226141,NaN,4.471524e+06,59.500000,58.129000,59.500000,58.260000,11367.0,XOM.N,2021-05-25
226142,NaN,4.434068e+06,59.115000,57.900000,58.350000,58.940000,11470.0,XOM.N,2021-05-26
226143,NaN,1.117998e+07,59.700000,58.030000,59.240000,58.560000,11667.0,XOM.N,2021-05-27
226144,NaN,5.927672e+06,58.775000,58.115000,58.710000,58.370000,10677.0,XOM.N,2021-05-28
